In [1]:
# ------------------------------------------define logging and working directory
from ProjectRoot import change_wd_to_project_root
change_wd_to_project_root()
from src.utils.Tensorflow_helper import choose_gpu_by_id
# ------------------------------------------define GPU id/s to use, if given
GPU_IDS = '0,1'
GPUS = choose_gpu_by_id(GPU_IDS)
print(GPUS)
# ------------------------------------------jupyter magic config
%matplotlib inline
%reload_ext autoreload
%autoreload 2
# ------------------------------------------ import helpers
# this should import glob, os, and many other standard libs
# local imports
from src.utils.Notebook_imports import *
from src.utils.Utils_io import Console_and_file_logger, init_config

# import external libs
from tensorflow.python.client import device_lib
import tensorflow as tf
tf.get_logger().setLevel('ERROR')
import cv2
import pandas as pd

EXPERIMENT = 'cv_baseline_test/sub-identifier'
timestemp = str(datetime.datetime.now().strftime("%Y-%m-%d_%H_%M")) # ad a timestep to each project to make repeated experiments unique

EXPERIMENTS_ROOT = 'exp/'
EXP_PATH = os.path.join(EXPERIMENTS_ROOT, EXPERIMENT, timestemp)
MODEL_PATH = os.path.join(EXP_PATH, 'model', )
TENSORBOARD_PATH = os.path.join(EXP_PATH, 'tensorboard_logs')
CONFIG_PATH = os.path.join(EXP_PATH,'config')
HISTORY_PATH = os.path.join(EXP_PATH, 'history')


# define the input data paths and fold 
# first to the 4D Nrrd files, 
# second to a dataframe with a mapping of the Fold-number
# Finally the path to the metadata
DATA_PATH_SAX = 'path_to_cmr_images'
DF_FOLDS = 'path_to_patient_fold_mapping'
DF_META = 'path_to_df_with_patient_metadata'
FOLD = 0

# General params
SEED = 42 # define a seed for the generator shuffle
BATCHSIZE = 8 # 32, 64, 24, 16, 1 depends on the input shape and available memory
GENERATOR_WORKER = BATCHSIZE # if not set, use batchsize
EPOCHS = 50 # define the max number of epochs to train on

DIM = [224, 224] # network input shape for spacing of 3, (z,y,x)
SPACING = [1.5,1.5] # if resample, resample to this spacing, (z,y,x)

# Model params
DEPTH = 4 # depth of the encoder/decoder architecture
FILTERS = 32 # initial number of filters, will be doubled after each downsampling block
M_POOL = [2, 2]# size of max-pooling used for downsampling and upsampling
F_SIZE = [3, 3] # conv filter size
BN_FIRST = False # decide if batch normalisation between conv and activation or afterwards
BATCH_NORMALISATION = True # apply BN or not
PAD = 'same' # padding strategy of the conv layers
KERNEL_INIT = 'he_normal' # conv weight initialisation
OPTIMIZER = 'adam' # Adam, Adagrad, RMSprop, Adadelta,  # https://keras.io/optimizers/
ACTIVATION = 'relu' # tf.keras.layers.LeakyReLU(), relu or any other non linear activation function
LEARNING_RATE = 1e-4 # start with a huge lr to converge fast
REDUCE_LR_ON_PLAEAU_PATIENCE = 5
DECAY_FACTOR = 0.7 # Define a learning rate decay for the ReduceLROnPlateau callback
MIN_LR = 1e-12 # minimal lr, smaller lr does not improve the model
DROPOUT_min = 0.3 # lower dropout at the shallow layers
DROPOUT_max = 0.5 # higher dropout at the deep layers

# Callback params
MONITOR_FUNCTION = 'loss'
MONITOR_MODE = 'min'
SAVE_MODEL_FUNCTION = 'loss'
SAVE_MODEL_MODE = 'min'
MODEL_PATIENCE = 20
SAVE_LEARNING_PROGRESS_AS_TF = True

# Generator and Augmentation params
BORDER_MODE = cv2.BORDER_REFLECT_101 # border mode for the data generation
IMG_INTERPOLATION = cv2.INTER_LINEAR # image interpolation in the genarator
MSK_INTERPOLATION = cv2.INTER_NEAREST # mask interpolation in the generator
AUGMENT = True # a compose of 2D augmentation (grid distortion, 90degree rotation, brightness and shift)
AUGMENT_PROB = 0.8
SHUFFLE = True
RESAMPLE = True
SCALER = 'MinMax' # MinMax, Standard or Robust

config = init_config(config=locals(), save=True)

search for root_dir and set working directory
Working directory set to: /mnt/ssd/git/wft21_septum_landmark_detection
['/gpu:0', '/gpu:1']


In [2]:
from src.models.train_model import train_fold

folds = [i for i in range(0, 4, 1)]

for f in folds:
    info('starting fold: {}'.format(f))
    config_ = config.copy()
    config_['FOLD'] = f
    train_fold(config_)

DEBUG:root:Creating directory /mnt/ssd/git/dynamic-cmr-models/logs/cv_baseline_taug10
2021-03-04 00:40:00,059 INFO -------------------- Start --------------------
2021-03-04 00:40:00,060 INFO Working directory: /mnt/ssd/git/dynamic-cmr-models.
2021-03-04 00:40:00,060 INFO Log file: ./logs/cv_baseline_taug10/8_64_64__8_3_3_4tenc_conv1_MSE_NOnorm_augshiftrot_NOtaug_3_batch8_f0.log
2021-03-04 00:40:00,060 INFO Log level for console: INFO
2021-03-04 00:40:00,061 INFO Is built with tensorflow: True
2021-03-04 00:40:00,110 INFO Visible devices:
[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:XLA_CPU:0', device_type='XLA_CPU'), PhysicalDevice(name='/physical_device:XLA_GPU:0', device_type='XLA_GPU'), PhysicalDevice(name='/physical_device:XLA_GPU:1', device_type='XLA_GPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]
2021-03-04 00:40:00,783 INFO Local

['/gpu:0', '/gpu:1']
{'GPU_IDS': '0,1', 'GPUS': ['/gpu:0', '/gpu:1'], 'SEED': 42, 'EXPERIMENT': 'cv_baseline_taug10/8_64_64__8_3_3_4tenc_conv1_MSE_NOnorm_augshiftrot_NOtaug_3_batch8_f0', 'EXPERIMENTS_ROOT': 'exp/', 'EXP_PATH': 'exp/cv_baseline_taug10/8_64_64__8_3_3_4tenc_conv1_MSE_NOnorm_augshiftrot_NOtaug_3_batch8_f0/2021-03-04_00_40', 'MODEL_PATH': 'exp/cv_baseline_taug10/8_64_64__8_3_3_4tenc_conv1_MSE_NOnorm_augshiftrot_NOtaug_3_batch8_f0/2021-03-04_00_40/model', 'TENSORBOARD_PATH': 'exp/cv_baseline_taug10/8_64_64__8_3_3_4tenc_conv1_MSE_NOnorm_augshiftrot_NOtaug_3_batch8_f0/2021-03-04_00_40/tensorboard_logs', 'CONFIG_PATH': 'exp/cv_baseline_taug10/8_64_64__8_3_3_4tenc_conv1_MSE_NOnorm_augshiftrot_NOtaug_3_batch8_f0/2021-03-04_00_40/config', 'HISTORY_PATH': 'exp/cv_baseline_taug10/8_64_64__8_3_3_4tenc_conv1_MSE_NOnorm_augshiftrot_NOtaug_3_batch8_f0/2021-03-04_00_40/history', 'DATA_PATH_SAX': '/mnt/ssd/data/gcn/02_imported_4D_unfiltered/SAX/', 'DF_FOLDS': '/mnt/ssd/data/gcn/02_importe

2021-03-04 00:40:00,785 INFO no files found, try to load with clean.nrrd/mask.nrrd pattern
2021-03-04 00:40:00,793 INFO Found 278 images/masks in /mnt/ssd/data/gcn/02_imported_4D_unfiltered/SAX/
2021-03-04 00:40:00,800 INFO Patients train: 209
2021-03-04 00:40:00,812 INFO Selected 209 of 278 files with 209 of 279 patients for training fold 0
2021-03-04 00:40:00,812 INFO SAX train CMR: 209, SAX train masks: 209
2021-03-04 00:40:00,813 INFO SAX val CMR: 69, SAX val masks: 69
2021-03-04 00:40:00,826 INFO Check if we find the patient ID and phase mapping for all: 278 files.
2021-03-04 00:40:01,093 INFO Done!
2021-03-04 00:40:01,094 INFO Create DataGenerator
2021-03-04 00:40:01,096 INFO Datagenerator created with: 
 shape: [8, 64, 64]
 spacing: [8, 3, 3]
 batchsize: 8
 Scaler: MinMax
 Images: 209 
 Augment: True 
 Thread workers: 8
2021-03-04 00:40:01,096 INFO Data will be augmented (shift,scale and rotate) with albumentation
2021-03-04 00:40:01,109 INFO Smoothing kernel: 
[ 1.   1.8  2.6  

Shape after the temporal encoder
(None, 36, 8, 4, 4, 512)
Shape after GAP
(None, 36, 512)
Shape after Bi-LSTM layer
(None, 36, 512)
Shape after final conv layer
(None, 36, 5)


2021-03-04 00:40:05,515 INFO feed 4 Tensorboard is ready


...


Epoch 00048: loss did not improve from 0.39167
26/26 [==============================] - 34s 1s/step - loss: 0.3984 - mse_wrapper: 0.3984 - ca_wrapper: 0.8316 - meandiff: 3.0288 - val_loss: 0.8109 - val_mse_wrapper: 0.8109 - val_ca_wrapper: 0.7700 - val_meandiff: 4.5938 - lr: 1.0000e-04
Epoch 49/50
26/26 [==============================] - ETA: 0s - loss: 0.3808 - mse_wrapper: 0.3808 - ca_wrapper: 0.8341 - meandiff: 3.0769
Epoch 00049: loss improved from 0.39167 to 0.38079, saving model to exp/cv_baseline_taug10/8_64_64__8_3_3_4tenc_conv1_MSE_NOnorm_augshiftrot_NOtaug_3_batch8_f3/2021-03-04_02_09/model/model.h5
26/26 [==============================] - 33s 1s/step - loss: 0.3808 - mse_wrapper: 0.3808 - ca_wrapper: 0.8341 - meandiff: 3.0769 - val_loss: 0.5763 - val_mse_wrapper: 0.5763 - val_ca_wrapper: 0.8086 - val_meandiff: 4.2500 - lr: 1.0000e-04
Epoch 50/50
26/26 [==============================] - ETA: 0s - loss: 0.3864 - mse_wrapper: 0.3864 - ca_wrapper: 0.8309 - meandiff: 2.9375

2021-03-04 02:39:48,729 INFO (4, 2, 36, 5)
2021-03-04 02:39:49,287 INFO (4, 2, 36, 5)



Epoch 00050: loss did not improve from 0.38079
26/26 [==============================] - 34s 1s/step - loss: 0.3864 - mse_wrapper: 0.3864 - ca_wrapper: 0.8309 - meandiff: 2.9375 - val_loss: 1.3529 - val_mse_wrapper: 1.3529 - val_ca_wrapper: 0.7396 - val_meandiff: 4.2812 - lr: 1.0000e-04


2021-03-04 02:39:51,639 INFO Fold 3 finished after 1815.806 sec


True